In [1]:
import pandas as pd
import numpy as np
import collections
import copy
import random
import re
import matplotlib.pyplot as plt
from nlp.utils import (
    plot_histogram,
    get_completetext,
    plot_wordcloud,
    print_statistics,
    groups_frequency_sort)
from nlp.text_statistics import (
    count_tokens,
    unique_tokens
)
from utils.read_files import (
    get_items)
from item.item_list import (
    ItemList,
    Item
)
from item.utils import get_tokens_set
import seaborn as sns
from item.clustering.utils import (
    load_clustering_results_pickle,
    load_clustering_results,
    get_clusters_dataframe,
    get_items_dataframe,
    add_first_token_column,
    add_outlier_column
)

sns.set()

# Load items

In [2]:
# It gets the descriptions processed:
itemlist = ItemList()
itemlist.load_items_from_file('../data/items.csv.zip')

In [3]:
len(itemlist.items_df)

2149533

# Load results

In [4]:
results, outliers = load_clustering_results_pickle('/scratch/pedrobrum/masters/data/clustering/hdbscan_bert/')

In [5]:
cluster_items_df = get_clusters_dataframe(results, baseline=True)

In [6]:
cluster_items_df.head()

,item_id,grupo,grupo_ruido,item_ruido
0,1504296,medendazol,1,1
1,1803013,medendazol,1,1
2,2010509,urupem,1,1
3,1187827,travaquedas,1,1
4,2036213,prednsolona,1,1


In [7]:
items_clusters_df = get_items_dataframe(itemlist.items_df, cluster_items_df)

In [8]:
items_clusters_df.head()

,item_id,grupo,grupo_ruido,item_ruido,palavras,unidades_medida,numeros,cores,materiais,tamanho,quantidade,original_prep,primeiro_termo
0,1504296,medendazol,1,1,['medendazol'],['mg'],['1.00e+02'],[],[],[],['comprimido'],"['medendazol', '1.00e+02', 'mg', 'comprimido']",medendazol
1,1803013,medendazol,1,1,"['medendazol', 'suspensao', 'doador', 'frasco']","['mg', 'ml']","['2.00e+01', '3.00e+01']",[],[],[],['frasco'],"['medendazol', '2.00e+01', 'mg', 'ml', 'suspen...",medendazol
2,2010509,urupem,1,1,"['urupem', 'disp', 'incontinencia', 'urinario']","['g', 'c']","['2.00e+00', '6.00e+00']",[],[],[],[],"['urupem', 'g', 'c', '2.00e+00', 'disp', 'inco...",urupem
3,1187827,travaquedas,1,1,"['travaquedas', 'para', 'corda', 'block', 'tra...",['mm'],"['1.10e+01', '1.20e+01']",[],[],[],[],"['travaquedas', 'para', 'corda', '1.10e+01', '...",travaquedas
4,2036213,prednsolona,1,1,"['prednsolona', 'frasco']","['mg', 'ml']","['1.00e+00', '1.00e+02']",[],[],[],['frasco'],"['prednsolona', '1.00e+00', 'mg', 'ml', 'frasc...",prednsolona


In [9]:
def get_statistics_df(items_clusters_df):

    group_by = ['grupo']
    results_df = items_clusters_df[group_by]
    results_df['n_items'] = results_df.groupby('grupo')['grupo'].transform('count')

    return results_df

In [10]:
len(items_clusters_df)

2050622

In [11]:
stats_df = get_statistics_df(items_clusters_df)

/home/speed/pedrobrum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [12]:
stats_df.head()

,grupo,n_items
0,medendazol,2
1,medendazol,2
2,urupem,1
3,travaquedas,1
4,prednsolona,1


In [13]:
stats_df = add_first_token_column(stats_df)

/scratch/pedrobrum/masters/src/item/clustering/utils.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['primeiro_termo'] = data['grupo'].str.split('_').str[0]


In [14]:
stats_df.head()

,grupo,n_items,primeiro_termo
0,medendazol,2,medendazol
1,medendazol,2,medendazol
2,urupem,1,urupem
3,travaquedas,1,travaquedas
4,prednsolona,1,prednsolona


In [15]:
stats_df[stats_df.primeiro_termo == "alcool"]

,grupo,n_items,primeiro_termo
1798755,alcool_6,1194,alcool
1798756,alcool_6,1194,alcool
1798757,alcool_6,1194,alcool
1798758,alcool_6,1194,alcool
1798759,alcool_6,1194,alcool
...,...,...,...
1804129,alcool_4,62,alcool
1804130,alcool_4,62,alcool
1804131,alcool_4,62,alcool
1804132,alcool_4,62,alcool


In [16]:
stats_df = stats_df.drop_duplicates(['grupo'])

In [17]:
len(stats_df)

36644

In [18]:
stats_df = add_outlier_column(stats_df)

In [19]:
stats_df = stats_df[stats_df.ruido != '-1']

## Print clusters info

In [20]:
def print_clusters_info(cluster_name, min_size=20, total_unit_metrics=5, total_clusters=5, total_items=5, total_words=20):

    clusters = stats_df[stats_df.primeiro_termo == cluster_name][['grupo', 'n_items']]
    clusters_ranking = clusters.sort_values(by='n_items', ascending=False)
    
    print("#itens:", sum(list(clusters["n_items"])))
    print("#grupos:", len(clusters_ranking))
    print("Grupos com +20 itens:", len(clusters_ranking[clusters_ranking.n_items >= min_size]))
    print("#itens/#grupos: {:.2f}".format(sum(list(clusters["n_items"]))/len(clusters_ranking)))
        
    clusters_ranking = clusters_ranking[clusters_ranking.n_items >= min_size].head(total_clusters)
    
    for cluster_id in clusters_ranking.index.values:
        cluster_info = clusters.loc[cluster_id]
        cluster_items = items_clusters_df[(items_clusters_df.grupo == cluster_info['grupo'])].copy()
        cluster_items["description_tokens"] = cluster_items.original_prep.apply(lambda d: eval(d))
        cluster_items["original"] = cluster_items.description_tokens.apply(lambda d: ' '.join(d))
        top_descriptions = cluster_items.groupby("original").size().sort_values(ascending=False).head(total_items)
        present_tokens = collections.Counter()
        for tokens in cluster_items["description_tokens"]:
            present_tokens.update(tokens)

        present_tokens = list(present_tokens.items())
        present_tokens.sort(key= lambda x : x[1], reverse=True)
        present_tokens = [token[0] for token in present_tokens][:total_words]

        print("Cluster {}: ".format(cluster_info['grupo']))
        print("Tamanho: {}".format(len(cluster_items)))
        print("Descrições únicas: {}".format(len(set(cluster_items["original"]))))
        print("{} Termos mais frequentes: {}".format(total_words, present_tokens))
        print("Descrições mais frequentes:")
        for desc in top_descriptions.iteritems():
            print("{} ({:.1f}%)".format(desc[0], (desc[1]/len(cluster_items))*100))
        print("\n")

# Abobora

In [47]:
print_clusters_info("abobora")

#itens: 78
#grupos: 2
Grupos com +20 itens: 2
#itens/#grupos: 39.00
Cluster abobora_0: 
Tamanho: 40
Descrições únicas: 37
20 Termos mais frequentes: ['abobora', 'com', 'kg', 'pesar', '2.00e+00', 'moranga', 'sem', 'japones', 'qualidade', 'apresentar', 'medio', 'firmar', 'cor', 'dano', 'tamanho', 'isentar', 'grau', 'produto', 'perfuracao', 'madurar']
Descrições mais frequentes:
abobora moranga kg produto fresco com grau maturacao completo apresentar cor polpa intensar odor agradavel conscistencia firmar isentar perfuracao machucado (7.5%)
abobora japones bom qualidade dever apresentar dano origem fisica mecanico biologico afetar aparencia pesar medio unidade 1.00e+00 kg variacao total (5.0%)
abobora tipo sergipano madurar com pesar minimo 2.00e+00 boi kg (2.5%)
abobora japones com casca verde integro ain natura apresentar grau maturacao permitir suportar manipulacao dever apresentar dano origem mecanico biologico (2.5%)
abobora japonesaabobora japones bem desenvolver compactar firmar apr

# Dipirona

In [29]:
print_clusters_info("dipirona")

#itens: 3379
#grupos: 321
Grupos com +20 itens: 44
#itens/#grupos: 10.53

Unidades de medida mais frequentes:
unidade (26.0%)
frasco (23.4%)
comprimido (19.1%)
ampola (12.2%)
amp (3.9%)

Cluster dipirona_9 (comprimido): 
Tamanho: 327
Descrições únicas: 22
20 Termos mais frequentes: ['dipirona', '500', 'mg', 'comprimido', 'comp', 'compr', 'cpr', 'cp', '129', '8974', '4731', '345', '0017', '959', '9', 'un', '966', 'epp', 'medic', 'fracionar']
Descrições mais frequentes:
dipirona 500 mg (60.6%)
dipirona 500 mg comprimido (15.0%)
dipirona 500 mg comp (7.3%)
dipirona 500 mg compr (3.1%)
dipirona 500 mg cpr (2.1%)


Cluster dipirona_2 (comprimido): 
Tamanho: 248
Descrições únicas: 18
20 Termos mais frequentes: ['dipirona', 'sodico', 'mg', '500', 'comprimido', 'comp', 'compr', 'cpr', 'cp', '9', '5', '250', 'com', 'ct', 'bl', 'al', 'plas', 'inc', 'x', '240']
Descrições mais frequentes:
dipirona sodico 500 mg (60.9%)
dipirona sodico 500 mg comprimido (23.0%)
dipirona sodico 500 mg comp (3.6%)
d

In [ ]:
dipirona = items_df[items_df.first_token == 'dipirona']

In [ ]:
set(dipirona[(dipirona.cluster == 'dipirona_9') & (dipirona.dsc_unidade_medida == 'comprimido')]['description'])

In [ ]:
dipirona[(dipirona.cluster == 'dipirona_0') & (dipirona.dsc_unidade_medida == 'ampola')]

# Pneu

In [15]:
print_clusters_info("pneu")

#itens: 43417
#grupos: 660
Grupos com +20 itens: 338
#itens/#grupos: 65.78

Unidades de medida mais frequentes:
unidade (95.0%)
peca (2.9%)
unidades (1.4%)
serv (0.2%)
pca (0.1%)

Cluster pneu_46 (unidade): 
Tamanho: 11916
Descrições únicas: 1997
20 Termos mais frequentes: ['pneu', 'r', '70', '75', '16', '15', '5', '205', 'x', 'r15', 'r14', '185', '225', '17', '215', '80', '65', 'r16', '195', '20']
Descrições mais frequentes:
pneu 175 70 r14 (2.9%)
pneu 215 75 r 17 5 (2.6%)
pneu 215 75 17 5 (2.4%)
pneu 205 70 r15 (1.9%)
pneu 205 70 r 15 (1.9%)


Cluster pneu_44 (unidade): 
Tamanho: 1868
Descrições únicas: 286
20 Termos mais frequentes: ['pneu', '14', 'r', '70', '175', '185', 'x', '65', 'c', '9', 'radial', 'lona', 't', '24', '28', '80', '8', '88', '20', '60']
Descrições mais frequentes:
pneu 175 70 r 14 (14.8%)
pneu 175 70 14 (7.9%)
pneu 185 r 14 (6.4%)
pneu 185 70 r 14 (6.3%)
pneu 175 65 r 14 (4.2%)


Cluster pneu_188 (unidade): 
Tamanho: 1727
Descrições únicas: 409
20 Termos mais freq

In [ ]:
pneu = items_df[items_df.first_token == 'pneu']

In [ ]:
set(pneu[(pneu.cluster == 'pneu_46') & (pneu.dsc_unidade_medida == 'unidade')]['description'])

In [ ]:
pneu[(pneu.cluster == 'pneu_1') & (pneu.dsc_unidade_medida == 'unidade')]

# Gasolina

In [21]:
print_clusters_info("gasolina")

#itens: 5448
#grupos: 35
Grupos com +20 itens: 17
#itens/#grupos: 155.66

Unidades de medida mais frequentes:
litro (97.2%)
unidade (2.3%)
litro s (0.1%)
21 (0.1%)
gab pref (0.1%)

Cluster gasolina_8 (litro): 
Tamanho: 3284
Descrições únicas: 34
20 Termos mais frequentes: ['gasolina', 'comum', 'automotivo', 'tipo', 'c', 'aditivado', 'aut', 'm', 'v', 'epp', 'r1', 'r', 'comumgasolina', 'combust', 'incolor', 'amarelar', 'social', 'amarelo', 'assist', 'r2']
Descrições mais frequentes:
gasolina comum (90.9%)
gasolina automotivo comum (3.7%)
gasolina automotivo tipo comum (1.1%)
gasolina c comum (0.7%)
gasolina comum automotivo (0.6%)


Cluster gasolina_5 (litro): 
Tamanho: 945
Descrições únicas: 20
20 Termos mais frequentes: ['gasolina', 'aditivado', 'automotivo', 'comun', 'c', 'tipo', 'activada', 'comuma', 'cras', 'sec', 'prom', 'social', 'n', 'common', 'montalvania', 'adm', 'f', 'grid']
Descrições mais frequentes:
gasolina (61.0%)
gasolina aditivado (19.2%)
gasolina comun (6.9%)
gasolina 

# Máscara

In [32]:
print_clusters_info("mascara")

#itens: 8007
#grupos: 349
Grupos com +20 itens: 87
#itens/#grupos: 22.94

Unidades de medida mais frequentes:
unidade (58.7%)
caixa (29.2%)
pacote (4.0%)
peca (1.8%)
kit (1.6%)

Cluster mascara_26 (caixa): 
Tamanho: 1304
Descrições únicas: 351
20 Termos mais frequentes: ['mascara', 'descartavel', 'elastico', '50', 'c', 'com', 'cirurgico', 'unidade', 'caixa', 'cx', '100', 'desc', 'unid', 'branco', 'descartar', 'und', 'un', 'tnt', 'cor', 'descarar']
Descrições mais frequentes:
mascara descartavel (11.4%)
mascara cirurgico descartavel (7.4%)
mascara cirurgico (2.5%)
mascara descartavel com elastico (2.5%)
mascara descartavel c elastico (2.4%)


Cluster mascara_26 (unidade): 
Tamanho: 925
Descrições únicas: 312
20 Termos mais frequentes: ['mascara', 'descartavel', 'com', 'cirurgico', 'elastico', '50', 'a95', 'n', '95', 'c', 'nebulizacao', 'unidade', 'pff', 'p', 'azul', 'respirador', 'respiratorio', 'caixa', 'facial', 'venturi']
Descrições mais frequentes:
mascara descartavel (9.2%)
mascara

# Locação

In [34]:
print_clusters_info("locacao")

#itens: 10733
#grupos: 1596
Grupos com +20 itens: 122
#itens/#grupos: 6.72

Unidades de medida mais frequentes:
unidade (38.1%)
diaria (11.6%)
hora (7.5%)
sv (3.4%)
mes (3.0%)

Cluster locacao_38 (unidade): 
Tamanho: 207
Descrições únicas: 18
20 Termos mais frequentes: ['banheiro', 'quimico', 'locacao', 'unidade', 'individual', 'equipamento', '978', '3939', 'polietileno', 'portatil', 'para', 'p', 'n', 'adaptado', 'carnaval', 'unid', 'plastico', 'def', 'deficiente', '4183']
Descrições mais frequentes:
locacao banheiro quimico (76.8%)
locacao banheiro quimico unidade (4.8%)
locacao banheiro quimico individual (2.4%)
locacao banheiro quimico 3939 (1.9%)
locacao banheiro quimico 978 (1.9%)


Cluster locacao_22 (unidade): 
Tamanho: 194
Descrições únicas: 50
20 Termos mais frequentes: ['locacao', 'tenda', 'x', '6', 'm', '4', '5', 'barraca', '8', 'piramidal', 'lona', 'unidade', 'tipo', 'mts', 'tamanho', 'i', 'tensionado', 'mx', 'matalon', 'medir']
Descrições mais frequentes:
locacao tenda (25

# Veículo

In [35]:
print_clusters_info("veiculo")

#itens: 3722
#grupos: 180
Grupos com +20 itens: 44
#itens/#grupos: 20.68

Unidades de medida mais frequentes:
unidade (79.2%)
sv (3.7%)
km (3.5%)
hora (2.1%)
kilometro (2.0%)

Cluster veiculo_35 (unidade): 
Tamanho: 1104
Descrições únicas: 911
20 Termos mais frequentes: ['veiculo', 'minimo', 'porta', 'ano', '1', '4', 'km', 'eletrico', '0', 'com', 'cor', '5', 'direcao', 'hidraulico', 'branco', 'modelo', 'motor', 'capacidade', 'ar', 'zero']
Descrições mais frequentes:
veiculo novo zero quilometro capacidade para cinco passageiro quatro porta cor branco motor 1 0 flex fabricacao 2014 2015 (1.0%)
veiculo popular zero quilometro 4 quatro porta direcao hidraulico flex cor branco para programa bolsa familia (0.7%)
veiculo automotor transporte passageiro com seguinte caracteristico ano 2013 modelo zero km 5 cinco lugar incluido motorista cor solido branco motor 1 0 refrigeracao agua 4 cilindro linha injecao eletronico potencia minimo 78 cv 8 oito valvula transmissao manual quatro marcha frente

In [27]:
print_clusters_info("lampada", total_words=10)

#itens: 15473
#grupos: 33
Grupos com +20 itens: 33
#itens/#grupos: 468.88
Cluster lampada_7: 
Tamanho: 1475
Descrições únicas: 1472
10 Termos mais frequentes: ['lampada', 'w', '1.27e+02', 'v', 'x', 'fluorescente', 'compactar', '4.00e+01', '1.00e+02', '3.40e+01']
Descrições mais frequentes:
lampada halogenar 1.00e+02 w 1.27e+02 v 2.02e+03 (0.2%)
lampada compactar 3.40e+01 w x 1.27e+02 v 2.02e+03 (0.1%)
lampada vela liso a27 4.00e+01 w 1.27e+02 v leitoso (0.1%)
lampada eletronico espiral 1.27e+02 v 3.20e+01 w (0.1%)
lampada eletronico compactar fluorescente 1.27e+02 v 3.00e+01 w (0.1%)


Cluster lampada_22: 
Tamanho: 1376
Descrições únicas: 1363
10 Termos mais frequentes: ['lampada', '1.20e+01', 'v', 'w', 'polo', '1.00e+00', 'led', '1.10e+02', '2.00e+00', '2.01e+03']
Descrições mais frequentes:
lampada 1.14e+03 1.20e+01 v fiat unir 2.01e+03 (0.4%)
lampada halogenar benon 2.00e+00 5.00e+00 v para rotoscopia fibra otica mini 3.00e+03 x 1.00e+00 8.80e+01 1.05e+02 dhl 2.50e+00 volt modelo (0